In [1]:
# import torch
# from peft import AutoPeftModelForCausalLM
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import pandas as pd
import json

import pandas as pd
import torch
from datasets import Dataset
from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
# from swanlab.integration.huggingface import SwanLabCallback
from transformers import DataCollatorForSeq2Seq, Trainer
import os
import re

/home/vipuser/anaconda3/envs/lr-sql/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = '/home/model/para_Qwen'
# lora_path = './final_checkpoint_newS_SFT_SQL_CN_TV'
# lora_path = './final_checkpoint_dataset1_SQL'
# lora_path = './final_checkpoint_newS_SFT_SQL_CN_TV'

In [3]:
# model = AutoPeftModelForCausalLM.from_pretrained(
#         lora_path, trust_remote_code= True, device_map='auto')
model = AutoModelForCausalLM.from_pretrained(
        model_path, trust_remote_code= True, device_map='auto')

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:22<00:00,  5.60s/it]


In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
val_dataset_path = "wiki/wikisql_test.csv"  
df = pd.read_csv(val_dataset_path)

In [7]:
print(df)

                                              question  \
0    what is the number of smallpox when typhoid fe...   
1            What teams had 9 in the top 5 and 1 wins?   
2         How many years were there with 348 attempts?   
3    What's the highest season number of an episode...   
4              What is the hand for 4 credits is 1600?   
..                                                 ...   
810  What are the total lanes that have a rank larg...   
811                       What is the start of lap 55?   
812  With a score of 70, this player's name is list...   
813  What is the average rank for a building with 2...   
814             Which mountain range has a rank of 19?   

                                          create_table  \
0    CREATE TABLE 1-1007688-1 (\n  "Year" TEXT,\n  ...   
1    CREATE TABLE 1-1012730-2 (\n  "Year" TEXT,\n  ...   
2    CREATE TABLE 1-1037590-1 (\n  "Year" TEXT,\n  ...   
3    CREATE TABLE 1-10718192-2 (\n  "No__in_series"...   
4    CREATE T

In [8]:
system_prompt = """
You are an intelligent SQL generation assistant.  
Your task is to generate a valid SQL query using the schema provided in <database>...</database>,  
based on the natural language request given in <question>...</question>.  

Return your answer strictly in the following format:  
<answer>YOUR_GENERATED_SQL</answer>  
"""
print(system_prompt)


You are an intelligent SQL generation assistant.  
Your task is to generate a valid SQL query using the schema provided in <database>...</database>,  
based on the natural language request given in <question>...</question>.  

Return your answer strictly in the following format:  
<answer>YOUR_GENERATED_SQL</answer>  



In [9]:
def make_user_prompt(question, database_schema):
    return f"<question>{question}</question>\n<database>{database_schema}</database>"

In [10]:
def make_conversation(question, database_schema):
    conversation = []
    conversation0 = {}
    conversation1 = {}
    conversation2 = {}
    user_prompt = make_user_prompt(question, database_schema)
    conversation0["role"] = "system"
    conversation0["content"] = system_prompt
    conversation1["role"] = "user"
    conversation1["content"] = user_prompt + "\n"
    conversation.append(conversation0)
    conversation.append(conversation1)
#     print(conversation2["content"])
    return conversation

In [11]:
def predict(messages, model, tokenizer):
    device = "cuda"
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=2048
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
#     print(response)
     
    return response

In [15]:
sample_list = []
correct_list = []
for index, row in df.iterrows():
    correct_sql = row["sql"] + " ;"
    correct_list.append(correct_sql)
    messages = make_conversation(row["question"],row["create_table"])
#     print(messages)
    print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    response = predict(messages, model, tokenizer)   
    print(response)
    m = re.search(r'(?s)(?<=<answer>).*?(?=</answer>)', response)  # DOTALL: . 可跨行
    sql = m.group(0) if m else response
    sample_list.append(sql)
    print(sql)
    print(correct_sql)
    print(f"<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next{index}>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<answer>SELECT "Smallpox" FROM 1-1007688-1 WHERE "Typhoid_fever" = '293'</answer>
SELECT "Smallpox" FROM 1-1007688-1 WHERE "Typhoid_fever" = '293'
SELECT MAX("Smallpox") FROM 1-1007688-1 WHERE "Typhoid_fever" = 293
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next0>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
SELECT `Team_s_` FROM 1-1012730-2 WHERE `Top_5` = '9' AND `Wins` = '1';
SELECT `Team_s_` FROM 1-1012730-2 WHERE `Top_5` = '9' AND `Wins` = '1';
SELECT "Team_s_" FROM 1-1012730-2 WHERE "Top_5" = 9 AND "Wins" = 1
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next1>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<answer>SELECT COUNT("Year") FROM 1-1037590-1 WHERE "Attempts" = '348'</answer>
SELECT

<answer>SELECT `Population_August_15__2012` FROM 1-12496904-1 WHERE `Population_Density_2012__km_2__` = '307'</answer>
SELECT `Population_August_15__2012` FROM 1-12496904-1 WHERE `Population_Density_2012__km_2__` = '307'
SELECT MAX("Population_August_15__2012") FROM 1-12496904-1 WHERE "Population_Density_2012__km_2__" = 307
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next21>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<answer>SELECT `Release_date` FROM 1-12588029-3 WHERE `Year` = '2003'</answer>
SELECT `Release_date` FROM 1-12588029-3 WHERE `Year` = '2003'
SELECT "Release_date" FROM 1-12588029-3 WHERE "Year" = 2003
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next22>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<answer>SELECT COUNT(*) FROM 1-12821159-14 WHERE _2nd > '2.0' AND Money_list_rank I

<answer>SELECT `Ends_Lost` FROM 1-1505809-2 WHERE `Blank_Ends` = '9'</answer>
SELECT `Ends_Lost` FROM 1-1505809-2 WHERE `Blank_Ends` = '9'
SELECT "Ends_Lost" FROM 1-1505809-2 WHERE "Blank_Ends" = 9
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next41>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
SELECT `Runner_up` FROM 1-1507431-1 WHERE `No_` = '2';
SELECT `Runner_up` FROM 1-1507431-1 WHERE `No_` = '2';
SELECT "Runner_up" FROM 1-1507431-1 WHERE "No_" = 2
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next42>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
SELECT "Segment_B" FROM 1-15187735-13 WHERE "Episode" = '167';
SELECT "Segment_B" FROM 1-15187735-13 WHERE "Episode" = '167';
SELECT "Segment_B" FROM 1-15187735-13 WHERE "Episode" = 167
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next43>>>>>>>>

<answer>SELECT COUNT(*) FROM 1-16654785-2 WHERE `Length__m_` = '455';</answer>
SELECT COUNT(*) FROM 1-16654785-2 WHERE `Length__m_` = '455';
SELECT COUNT("Name") FROM 1-16654785-2 WHERE "Length__m_" = 455
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next63>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
SELECT "Inns_of_Court_and_Chancery" FROM 1-16677738-1 WHERE "Without_the_Walls" = '70489';
SELECT "Inns_of_Court_and_Chancery" FROM 1-16677738-1 WHERE "Without_the_Walls" = '70489';
SELECT "Inns_of_Court_and_Chancery" FROM 1-16677738-1 WHERE "Without_the_Walls" = 70489
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next64>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
SELECT COUNT(*) FROM 1-1671401-2 WHERE `Top_10` = '5' AND `Top_5` BETWEEN 0 AND 10;
SELECT COUNT(*) FROM 1-1671401-2 WHERE `Top_10` = '

SELECT `Directed_by` FROM 1-18481791-3 WHERE `No__in_season` = '1';
SELECT `Directed_by` FROM 1-18481791-3 WHERE `No__in_season` = '1';
SELECT "Directed_by" FROM 1-18481791-3 WHERE "No__in_season" = 1
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next83>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
SELECT `Spokespersons` FROM 1-184803-4 WHERE `Voting_order` = '9';
SELECT `Spokespersons` FROM 1-184803-4 WHERE `Voting_order` = '9';
SELECT "Spokespersons" FROM 1-184803-4 WHERE "Voting_order" = 9
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next84>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
SELECT "Air_Force___Navy_score" FROM 1-1850339-2 WHERE "Season" = '1983';
SELECT "Air_Force___Navy_score" FROM 1-1850339-2 WHERE "Season" = '1983';
SELECT "Air_Force___Navy_score" FROM 1-1850339-2 WHERE "Season"

...

<answer>SELECT SUM("Points") FROM 2-1861430-3 WHERE "Year" = '1992'</answer>
SELECT SUM("Points") FROM 2-1861430-3 WHERE "Year" = '1992'
SELECT COUNT("Points") FROM 2-1861430-3 WHERE "Year" = 1992
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next721>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<answer>SELECT "Name" FROM 2-18662739-4 WHERE "Total" = '22'</answer>
SELECT "Name" FROM 2-18662739-4 WHERE "Total" = '22'
SELECT "Name" FROM 2-18662739-4 WHERE "Total" = 22
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next722>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<answer>SELECT `Water__sqmi_` FROM 2-18600760-6 WHERE `Pop___2010_` = '359' AND `Longitude` < '-97.176811'</answer>
SELECT `Water__sqmi_` FROM 2-18600760-6 WHERE `Pop___2010_` = '359' AND `Longitude` < '-97.176811'
SELECT MAX("Water__sqm

SELECT SUM("Wins") AS Total_Wins 
FROM 2-1238866-1 
WHERE "Games" < '7' AND "Goals_For" > '2' AND "Goals_Against" < '6' AND "Draws" = '1';
SELECT SUM("Wins") AS Total_Wins 
FROM 2-1238866-1 
WHERE "Games" < '7' AND "Goals_For" > '2' AND "Goals_Against" < '6' AND "Draws" = '1';
SELECT COUNT("Wins") FROM 2-1238866-1 WHERE "Games" < 7 AND "Goals_For" > 2 AND "Goals_Against" < 6 AND "Draws" = 1
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next741>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
SELECT `Wins` FROM 2-1226341-2 WHERE `Points` = '6' AND `Rank` = 'least';
SELECT `Wins` FROM 2-1226341-2 WHERE `Points` = '6' AND `Rank` = 'least';
SELECT MIN("Wins") FROM 2-1226341-2 WHERE "Points" = 6
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next742>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<answer>SEL

SELECT "Year" FROM 2-1226505-3 WHERE "Pts_" = '13';
SELECT "Year" FROM 2-1226505-3 WHERE "Pts_" = '13';
SELECT "Year" FROM 2-1226505-3 WHERE "Pts_" = 13
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next760>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<answer>SELECT `Capacity` FROM 2-12335018-1 WHERE `First_season_in_Segunda_Divisi_n` = '2013' AND `Top_division_titles` > '0' ORDER BY `Capacity` ASC LIMIT 1;</answer>
SELECT `Capacity` FROM 2-12335018-1 WHERE `First_season_in_Segunda_Divisi_n` = '2013' AND `Top_division_titles` > '0' ORDER BY `Capacity` ASC LIMIT 1;
SELECT MIN("Capacity") FROM 2-12335018-1 WHERE "First_season_in_Segunda_Divisi_n" = 2013 AND "Top_division_titles" > 0
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next761>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
SELECT AVG("Yards

SELECT `Points` FROM `2-13053979-1` WHERE `Draw` = '6' AND `Rank` < '10' ORDER BY `Points` ASC LIMIT 1;
SELECT `Points` FROM `2-13053979-1` WHERE `Draw` = '6' AND `Rank` < '10' ORDER BY `Points` ASC LIMIT 1;
SELECT MIN("Points") FROM 2-13053979-1 WHERE "Draw" = 6 AND "Rank" > 10
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next778>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<answer>SELECT `Ground` FROM 2-12666456-12 WHERE `Match` = '8';</answer>
SELECT `Ground` FROM 2-12666456-12 WHERE `Match` = '8';
SELECT "Ground" FROM 2-12666456-12 WHERE "Match" = 8
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next779>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<answer>SELECT `Gender` FROM 2-12303251-3 WHERE `Roll` = '627';</answer>
SELECT `Gender` FROM 2-12303251-3 WHERE `Roll` = '627';
SELECT "Gender" F

SELECT `College` FROM 2-13312898-46 WHERE `Pick__` = '1' AND `Overall` > '1' AND `Round` = '7';
SELECT `College` FROM 2-13312898-46 WHERE `Pick__` = '1' AND `Overall` > '1' AND `Round` = '7';
SELECT "College" FROM 2-13312898-46 WHERE "Pick__" = 1 AND "Overall" > 1 AND "Round" = 7
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next798>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<answer>SELECT MIN("Games") FROM 2-1241808-2 WHERE "Year" > '2013'</answer>
SELECT MIN("Games") FROM 2-1241808-2 WHERE "Year" > '2013'
SELECT MIN("Games") FROM 2-1241808-2 WHERE "Year" > 2013
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next799>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<answer>SELECT AVG(`Change____`) FROM 2-1260656-1 WHERE `Land_area__km__` = '546.74' AND `Population_density__per_km__` > '0.5'</answer

In [16]:
df_out = pd.DataFrame({
    "generated_sql": sample_list,
    "correct_sql": correct_list
})

# 保存为 CSV
df_out.to_csv("sql_comparison_Qwen2.csv", index=False, encoding="utf-8-sig")

print("已保存到 sql_comparison.csv")

已保存到 sql_comparison.csv
